In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2cents-task-2/DOGEUSDT_1h.csv
/kaggle/input/2cents-task-2/ETHUSDT_1h.csv
/kaggle/input/2cents-task-2/SOLUSDT_15m.csv
/kaggle/input/2cents-task-2/BNBUSDT_1d.csv
/kaggle/input/2cents-task-2/BNBUSDT_15m.csv
/kaggle/input/2cents-task-2/DOGEUSDT_1d.csv
/kaggle/input/2cents-task-2/BTCUSDT_15m.csv
/kaggle/input/2cents-task-2/SOLUSDT_1d.csv
/kaggle/input/2cents-task-2/DOGEUSDT_15m.csv
/kaggle/input/2cents-task-2/BTCUSDT_1d.csv
/kaggle/input/2cents-task-2/ETHUSDT_1d.csv
/kaggle/input/2cents-task-2/BNBUSDT_1h.csv
/kaggle/input/2cents-task-2/BTCUSDT_1h.csv
/kaggle/input/2cents-task-2/SOLUSDT_1h.csv
/kaggle/input/2cents-task-2/ETHUSDT_15m.csv


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load dataset
file_path = '/kaggle/input/2cents-task-2/SOLUSDT_1d.csv'  # Adjust to the correct file path for each dataset
data = pd.read_csv(file_path)

# Convert 'timestamp' to datetime and set it as index
data['timestamp'] = pd.to_datetime(data['timestamp'])
data.set_index('timestamp', inplace=True)

# Step 1: Moving Averages for Market Environment
short_window = 50
long_window = 200
data['short_mavg'] = data['close'].rolling(window=short_window, min_periods=1).mean()
data['long_mavg'] = data['close'].rolling(window=long_window, min_periods=1).mean()

# Market environment detection
data['market_env'] = np.where(data['short_mavg'] > data['long_mavg'], 'bull', 
                              np.where(data['short_mavg'] < data['long_mavg'], 'bear', 'sideways'))

# Step 2: RSI for Bull and Sideways Markets
def rsi(close, window=14):
    delta = close.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

data['rsi'] = rsi(data['close'])

# Step 3: Bollinger Bands for Bear Market
def bollinger_bands(close, window=20):
    middleband = close.rolling(window=window).mean()
    stddev = close.rolling(window=window).std()
    upperband = middleband + (stddev * 2)
    lowerband = middleband - (stddev * 2)
    return upperband, middleband, lowerband

data['upperband'], data['middleband'], data['lowerband'] = bollinger_bands(data['close'])

# Step 4: ATR for Volatility-based Stop Loss and Take Profit
def atr(high, low, close, window=14):
    high_low = high - low
    high_close = np.abs(high - close.shift())
    low_close = np.abs(low - close.shift())
    true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    return true_range.rolling(window=window).mean()

data['atr'] = atr(data['high'], data['low'], data['close'])

# Step 5: Strategy Logic Based on Market Environment
def apply_strategy(row, position, capital, price, stop_loss, take_profit):
    if row['market_env'] == 'bull':
        # Buy on dips in bull market (RSI < 30), sell when RSI > 70
        if row['rsi'] < 30 and position == 0:
            units = capital / price
            position += units
            capital -= units * price
            stop_loss = price - row['atr'] * 2
            take_profit = price + row['atr'] * 3
        elif row['rsi'] > 70 and position > 0:
            capital += position * price
            position = 0
    elif row['market_env'] == 'bear':
        # Use Bollinger Bands in bear market
        if row['close'] < row['lowerband'] and position == 0:
            units = capital / price
            position += units
            capital -= units * price
            stop_loss = price - row['atr'] * 2
            take_profit = price + row['atr'] * 3
        elif row['close'] > row['upperband'] and position > 0:
            capital += position * price
            position = 0
    elif row['market_env'] == 'sideways':
        # Buy near support (RSI < 40), sell near resistance (RSI > 60)
        if row['rsi'] < 40 and position == 0:
            units = capital / price
            position += units
            capital -= units * price
        elif row['rsi'] > 60 and position > 0:
            capital += position * price
            position = 0

    # Implement stop-loss and take-profit logic
    if position > 0 and price < stop_loss:
        capital += position * price
        position = 0
    elif position > 0 and price > take_profit:
        capital += position * price
        position = 0

    return position, capital, stop_loss, take_profit

# Step 6: Backtesting the Strategy
initial_capital = 100000.0
portfolio = pd.DataFrame(index=data.index)
portfolio['cash'] = initial_capital
portfolio['holdings'] = 0.0
portfolio['total'] = portfolio['cash']

# Initialize position and capital for backtesting
position = 0
capital = initial_capital
stop_loss = 0
take_profit = 0

# Apply the strategy over time
for index, row in data.iterrows():
    price = row['close']
    position, capital, stop_loss, take_profit = apply_strategy(row, position, capital, price, stop_loss, take_profit)
    
    # Track holdings and portfolio value
    portfolio.loc[index, 'cash'] = capital
    portfolio.loc[index, 'holdings'] = position * price
    portfolio.loc[index, 'total'] = portfolio['cash'] + portfolio['holdings']

# Step 7: Performance Reporting and Visualization
# Plot equity curve
plt.figure(figsize=(14, 8))
plt.plot(portfolio['total'], label='Total Portfolio Value')
plt.title('Equity Curve')
plt.xlabel('Date')
plt.ylabel('Portfolio Value (USD)')
plt.legend()
plt.grid()
plt.show()

# Print final capital and profit
final_capital = portfolio['total'][-1]
profit = final_capital - initial_capital
print(f"Final Portfolio Value: ${final_capital:.2f}")
print(f"Total Profit: ${profit:.2f}")


ValueError: Incompatible indexer with Series